In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import TimeoutException 


import pickle
from tqdm import tqdm

from explicit import waiter, XPATH
import time
import pandas as pd
from bs4 import BeautifulSoup
import json

In [2]:
def scrape_twitter(driver,accs):
    global master_dict
    master_dict = dict()
    iter_counter = 0
    
    for username in tqdm(accs):
        if iter_counter>50:
            iter_counter = 0 
            time.sleep(2)
        iter_counter+=1
        
        try:
            
            driver.get(f"https://twitter.com/{username}")
            
            try:
                WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, "(//span[text()='Follow']/ancestor::div[@role='button'])[1]")))
            except:
                WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, "(//span[text()='Following']/ancestor::div[@role='button'])[1]")))
                
            html_source = driver.page_source
            sourcedata = html_source.encode('utf-8')
            soup = BeautifulSoup( sourcedata , 'html.parser')


            name = soup.find_all('div',{"class":"css-901oao r-1awozwy r-1nao33i r-6koalj r-37j5jr r-adyw6z r-1vr29t4 r-135wba7 r-bcqeeo r-1udh08x r-qvutc0"})
            for i in name:
                name = i.find('span',{"class":"css-901oao css-16my406 r-poiln3 r-bcqeeo r-qvutc0"}).text

            screenname = soup.find_all('div',{"class":"css-901oao css-1hf3ou5 r-1bwzh9t r-18u37iz r-37j5jr r-a023e6 r-16dba41 r-rjixqe r-bcqeeo r-qvutc0"})
            screenname = screenname[0].find('span',{"class":"css-901oao css-16my406 r-poiln3 r-bcqeeo r-qvutc0"}).text

            user_id = soup.find("img",{"alt":'Opens profile photo'})["src"][37:-21]

            fo_fol = soup.find_all('span',{"class":"css-901oao css-16my406 r-1nao33i r-poiln3 r-1b43r93 r-b88u0q r-1cwl3u0 r-bcqeeo r-qvutc0"})

            following_count = fo_fol[0].find("span",{'class':"css-901oao css-16my406 r-poiln3 r-bcqeeo r-qvutc0"}).text
            followers_count = fo_fol[1].find("span",{'class':"css-901oao css-16my406 r-poiln3 r-bcqeeo r-qvutc0"}).text


            is_private = False
            if soup.find("div",{"class":"css-901oao r-1nao33i r-37j5jr r-1yjpyg1 r-1vr29t4 r-ueyrd6 r-5oul0u r-bcqeeo r-fdjqy7 r-qvutc0"}):
                is_private = True


            is_verified = False
            if soup.find("svg",{"aria-label":"Verified account"}):
                is_verified = True

            tweet_count = soup.find('div',{"class":"css-901oao css-1hf3ou5 r-1bwzh9t r-37j5jr r-n6v787 r-16dba41 r-1cwl3u0 r-bcqeeo r-qvutc0"}).text.replace("Tweets","").strip()

            is_bussiness_acc = False ; bussiness_sector = "NULL"
            if soup.find("span",{"data-testid":"UserProfessionalCategory"}):
                __ = soup.find("span",{"data-testid":"UserProfessionalCategory"})
                bussiness_sector = __.find("span",{'class':'css-901oao css-16my406 r-poiln3 r-bcqeeo r-qvutc0'}).text
                is_bussiness_acc = True

            location = "NULL"
            if soup.find("span",{"data-testid":"UserLocation"}):
                __ = soup.find("span",{"data-testid":"UserLocation"})
                location = __.find("span",{'class':'css-901oao css-16my406 r-poiln3 r-bcqeeo r-qvutc0'}).text

            bio_link = "NULL"
            if soup.find("a",{"data-testid":"UserUrl"}):
                bio_link = soup.find("a",{"data-testid":"UserUrl"})['href']

            date_joined = "NULL"
            if soup.find("span",{"data-testid":"UserJoinDate"}):
                __ = soup.find("span",{"data-testid":"UserJoinDate"})
                date_joined = __.find("span",{"class":"css-901oao css-16my406 r-poiln3 r-bcqeeo r-qvutc0"}).text


            user_bio = "NULL"
            if soup.find("div",{"data-testid":"UserDescription"}):
                user_bio = ""
                __ = soup.find_all("div",{"data-testid":"UserDescription"})
                user_bio = [user_bio+i.text for i in __][0]

            data = {

                "user id" : user_id,
                "screen name" : screenname,
                "name" : name,
                "followers" : followers_count,
                "following" : following_count,
                "bio" : user_bio,
                "bio link" : bio_link,
                "date joined" : date_joined,
                "location" : location,
                "is private" : is_private,
                "is verified" : is_verified,
                "is bussiness account" : is_bussiness_acc,
                "bussiness sector" : bussiness_sector,

            }
            master_dict.update({screenname:data})
            
            tdf = pd.DataFrame(data,index=[0])
            global df
            df = pd.concat([df,tdf], ignore_index = True)
        
        except TimeoutException:
            df.to_excel("twitter_scraped.xlsx")
            pass
        except IndexError:
            mdf.to_excel("twitter_scraped.xlsx")
            pass
        except TypeError:
            df.to_excel("twitter_scraped.xlsx")
            pass
        finally:
            df.to_excel("twitter_scraped.xlsx")

In [3]:
def scrape_twitter_daddy(driver,accs,acc_name):
    global master_dict
    master_dict = dict()
    iter_counter = 0
    
    for username in tqdm(accs):
        if iter_counter>10:
            iter_counter = 0 
            time.sleep(2)
        iter_counter+=1
        
        driver.get(f"https://twitter.com/{username}")

        try:
            
            WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.CSS_SELECTOR, "script[data-testid='UserProfileSchema-test']")))
        
        except TimeoutException:
            continue
        
        else:
            html_source = driver.page_source
            sourcedata = html_source.encode('utf-8')
            
            global soup
            soup = BeautifulSoup( sourcedata , 'html.parser')
                
            js = soup.find("script",{"data-testid":"UserProfileSchema-test"}).text

            js = js.replace("\n","")
            res = json.loads(js)

            user_id = res['author']['identifier']
            screenname = res['author']['additionalName']
            try:
                name = res['author']['givenName']
            except:
                name = "NULL"
            followers_count = res['author']['interactionStatistic'][0]['userInteractionCount']
            following_count = res['author']['interactionStatistic'][1]['userInteractionCount']
            tweet_count = res['author']['interactionStatistic'][2]['userInteractionCount']
            try:
                user_bio = res['author']['description']
                if not user_bio:
                    user_bio = "NULL"
            except:
                user_bio = "NULL"
            try:
                bio_link = res['relatedLink'][1]
            except:
                bio_link = "NULL"
            date_joined = res['dateCreated']
            try:
                location = res['author']['homeLocation']['name']
                if not location:
                    location = "NULL"
            except:
                location = "NULL"

            is_private = 'False'
            if soup.find("div",{"class":"css-901oao r-1nao33i r-37j5jr r-1yjpyg1 r-1vr29t4 r-ueyrd6 r-5oul0u r-bcqeeo r-fdjqy7 r-qvutc0"}):
                is_private = 'True'


            is_verified = 'False'
            if soup.find("svg",{"aria-label":"Verified account"}):
                is_verified = 'True'

            is_bussiness_acc = 'False' ; bussiness_sector = "NULL"
            if soup.find("span",{"data-testid":"UserProfessionalCategory"}):
                __ = soup.find("span",{"data-testid":"UserProfessionalCategory"})
                bussiness_sector = __.find("span",{'class':'css-901oao css-16my406 r-poiln3 r-bcqeeo r-qvutc0'}).text
                is_bussiness_acc = 'True'

            data = {
                
                "follower of" :acc_name,
                "user id" : user_id,
                "screen name" : screenname,
                "name" : name,
                "followers" : followers_count,
                "following" : following_count,
                "bio" : user_bio,
                "bio link" : bio_link,
                "date joined" : date_joined,
                "location" : location,
                "is private" : is_private,
                "is verified" : is_verified,
                "is bussiness account" : is_bussiness_acc,
                "bussiness sector" : bussiness_sector,

            }
            master_dict.update({screenname:data})

            tdf = pd.DataFrame(data,index=[0])
            global df
            df = pd.concat([df,tdf], ignore_index = True)
            df.to_excel(f"phase2/{acc_name}-phase2.xlsx")

In [59]:
username = "EdgeForex16"
password = "Success01!"

In [60]:
def login(driver):
    driver.get("https://twitter.com/i/flow/login")
    waiter.find_write(driver, "//*[@id='layers']/div/div/div/div/div/div/div[2]/div[2]/div/div/div[2]/div[2]/div/div/div/div[5]/label/div/div[2]/div/input", username, by=XPATH)
    waiter.find_element(driver, "//*[@id='layers']/div/div/div/div/div/div/div[2]/div[2]/div/div/div[2]/div[2]/div/div/div/div[6]/div", by=XPATH).click()
    waiter.find_write(driver, "//*[@id='layers']/div/div/div/div/div/div/div[2]/div[2]/div/div/div[2]/div[2]/div[1]/div/div/div[3]/div/label/div/div[2]/div[1]/input", password, by=XPATH)
    waiter.find_element(driver, "//*[@id='layers']/div/div/div/div/div/div/div[2]/div[2]/div/div/div[2]/div[2]/div[2]/div/div[1]/div/div/div/div", by=XPATH).click()
    time.sleep(4)

In [4]:
result_dict = dict()

options = Options()

options.headless = False
options.add_argument("--incognito")
driver = webdriver.Chrome(options=options,service=Service(ChromeDriverManager().install())) 
driver.get(f"https://www.twitter.com/")
cookies = pickle.load(open("twittercookies.pkl", "rb"))
for cookie in cookies:
    driver.add_cookie(cookie)

[WDM] - Current google-chrome version is 105.0.5195
[WDM] - Get LATEST driver version for 105.0.5195
[WDM] - Driver [C:\Users\sasiv\.wdm\drivers\chromedriver\win32\105.0.5195.52\chromedriver.exe] found in cache


In [5]:
sample_df = pd.read_excel("sample.xlsx")
accs = list(sample_df['Followers Usernames'].to_list())
accs = accs[145:]

In [7]:
df = pd.DataFrame(columns = ['user id', 'screen name', 'name', 'followers', 'following', 'bio', 'bio link', 'date joined', 'location', 'is private', 'is verified', 'is bussiness account', 'bussiness sector'])

In [8]:
scrape_twitter_daddy(driver,accs)

  1%|▋                                                                                 | 6/772 [00:19<59:27,  4.66s/it]

boo


 11%|████████▉                                                                        | 85/772 [02:31<47:23,  4.14s/it]

boo


 34%|███████████████████████████▎                                                    | 264/772 [08:02<41:55,  4.95s/it]

...

 72%|█████████████████████████████████████████████████████████▊                      | 558/772 [25:11<17:24,  4.88s/it]

boo


 75%|████████████████████████████████████████████████████████████▏                   | 581/772 [26:09<14:13,  4.47s/it]

boo


100%|████████████████████████████████████████████████████████████████████████████████| 772/772 [32:54<00:00,  2.56s/it]


In [29]:
len(master_dict)

5

In [57]:
soup

<html dir="ltr" lang="en" style="overflow-y: scroll; overscroll-behavior-y: none; font-size: 15px;"><head><style>input::placeholder { user-select: none; -webkit-user-select: none; } iframe { color-scheme: auto; }</style><style>@font-face {
  font-family: TwitterChirpExtendedHeavy;
  src: url(https://abs.twimg.com/fonts/v1/chirp-extended-heavy-web.woff2) format('woff2');
  src: url(https://abs.twimg.com/fonts/v1/chirp-extended-heavy-web.woff) format('woff');
  font-weight: 800;
  font-style: 'normal';
  font-display: 'swap';
}
@font-face {
  font-family: TwitterChirp;
  src: url(https://abs.twimg.com/fonts/v2/chirp-regular-web.woff2) format('woff2');
  src: url(https://abs.twimg.com/fonts/v2/chirp-regular-web.woff) format('woff');
  font-weight: 400;
  font-style: 'normal';
  font-display: 'swap';
}
@font-face {
  font-family: TwitterChirp;
  src: url(https://abs.twimg.com/fonts/v2/chirp-medium-web.woff2) format('woff2');
  src: url(https://abs.twimg.com/fonts/v2/chirp-medium-web.woff) 

# Rough Work

In [69]:
username = "nuvoforex"

In [70]:
driver.get(f"https://twitter.com/{username}")
WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, "(//span[text()='Follow']/ancestor::div[@role='button'])[1]")))
html_source = driver.page_source
sourcedata = html_source.encode('utf-8')
soup = BeautifulSoup( sourcedata , 'html.parser')

In [71]:
name = soup.find_all('div',{"class":"css-901oao r-1awozwy r-1nao33i r-6koalj r-37j5jr r-adyw6z r-1vr29t4 r-135wba7 r-bcqeeo r-1udh08x r-qvutc0"})
for i in name:
    name = i.find('span',{"class":"css-901oao css-16my406 r-poiln3 r-bcqeeo r-qvutc0"}).text
name

'Edge Forex'

In [72]:
screenname = soup.find_all('div',{"class":"css-901oao css-1hf3ou5 r-1bwzh9t r-18u37iz r-37j5jr r-a023e6 r-16dba41 r-rjixqe r-bcqeeo r-qvutc0"})
screenname = screenname[0].find('span',{"class":"css-901oao css-16my406 r-poiln3 r-bcqeeo r-qvutc0"}).text
screenname

'@nuvoforex'

In [73]:
user_id = soup.find("img",{"alt":'Opens profile photo'})["src"][37:-21]
user_id

'1533791006021279745'

In [74]:
fo_fol = soup.find_all('span',{"class":"css-901oao css-16my406 r-1nao33i r-poiln3 r-1b43r93 r-b88u0q r-1cwl3u0 r-bcqeeo r-qvutc0"})

following_count = fo_fol[0].find("span",{'class':"css-901oao css-16my406 r-poiln3 r-bcqeeo r-qvutc0"}).text
followers_count = fo_fol[1].find("span",{'class':"css-901oao css-16my406 r-poiln3 r-bcqeeo r-qvutc0"}).text

print(followers_count,following_count)

1,224 4,952


In [75]:
is_private = False
if soup.find("div",{"class":"css-901oao r-1nao33i r-37j5jr r-1yjpyg1 r-1vr29t4 r-ueyrd6 r-5oul0u r-bcqeeo r-fdjqy7 r-qvutc0"}):
    is_private = True
is_private

False

In [76]:
is_verified = False
if soup.find("svg",{"aria-label":"Verified account"}):
    is_verified = True
is_verified

False

In [77]:
tweet_count = soup.find('div',{"class":"css-901oao css-1hf3ou5 r-1bwzh9t r-37j5jr r-n6v787 r-16dba41 r-1cwl3u0 r-bcqeeo r-qvutc0"}).text.replace("Tweets","").strip()
tweet_count

'3,946'

In [78]:
is_bussiness_acc = False ; bussiness_sector = "NULL"
if soup.find("span",{"data-testid":"UserProfessionalCategory"}):
    __ = soup.find("span",{"data-testid":"UserProfessionalCategory"})
    bussiness_sector = __.find("span",{'class':'css-901oao css-16my406 r-poiln3 r-bcqeeo r-qvutc0'}).text
    is_bussiness_acc = True

print(is_bussiness_acc,bussiness_sector)

True Financial Tech Company


In [79]:
location = "NULL"
if soup.find("span",{"data-testid":"UserLocation"}):
    __ = soup.find("span",{"data-testid":"UserLocation"})
    location = __.find("span",{'class':'css-901oao css-16my406 r-poiln3 r-bcqeeo r-qvutc0'}).text
location

'Toronto, Canada'

In [80]:
bio_link = "NULL"
if soup.find("a",{"data-testid":"UserUrl"}):
    bio_link = soup.find("a",{"data-testid":"UserUrl"})['href']
bio_link

'https://t.co/0GR1LP0SCS'

In [81]:
date_joined = "NULL"
if soup.find("span",{"data-testid":"UserJoinDate"}):
    __ = soup.find("span",{"data-testid":"UserJoinDate"})
    date_joined = __.find("span",{"class":"css-901oao css-16my406 r-poiln3 r-bcqeeo r-qvutc0"}).text
date_joined

'Joined September 2014'

In [82]:
user_bio = "NULL"
if soup.find("div",{"data-testid":"UserDescription"}):
    user_bio = ""
    __ = soup.find_all("div",{"data-testid":"UserDescription"})
    user_bio = [user_bio+i.text for i in __][0]
user_bio

"We are FOREX signal service provider,who will work hard to get you the best results,so you don't have to."

In [83]:
data = {

                "user id" : user_id,
                "screen name" : screenname,
                "name" : name,
                "followers" : followers_count,
                "following" : following_count,
                "bio" : user_bio,
                "bio link" : bio_link,
                "date joined" : date_joined,
                "location" : location,
                "is private" : is_private,
                "is verified" : is_verified,
                "is bussiness account" : is_bussiness_acc,
                "bussiness sector" : bussiness_sector,

            }

In [84]:
import json 



{
    "user id": "1533791006021279745",
    "screen name": "@nuvoforex",
    "name": "Edge Forex",
    "followers": "1,224",
    "following": "4,952",
    "bio": "We are FOREX signal service provider,who will work hard to get you the best results,so you don't have to.",
    "bio link": "https://t.co/0GR1LP0SCS",
    "date joined": "Joined September 2014",
    "location": "Toronto, Canada",
    "is private": false,
    "is verified": false,
    "is bussiness account": true,
    "bussiness sector": "Financial Tech Company"
}


In [92]:
js = soup.find("script",{"data-testid":"UserProfileSchema-test"}).text
js = js.replace("\n","")
res = json.loads(js)

In [100]:
res

{'@context': 'http://schema.org',
 '@type': 'ProfilePage',
 'dateCreated': '2014-09-28T11:32:37.000Z',
 'author': {'@type': 'Person',
  'additionalName': 'nuvoforex',
  'description': "We are FOREX signal service provider,who will work hard to get you the best results,so you don't have to.",
  'givenName': 'Edge Forex',
  'homeLocation': {'@type': 'Place', 'name': 'Toronto, Canada'},
  'identifier': '2788849409',
  'image': {'@type': 'ImageObject',
   'contentUrl': 'https://pbs.twimg.com/profile_images/1533791006021279745/5Qke2kf6_400x400.jpg',
   'thumbnailUrl': 'https://pbs.twimg.com/profile_images/1533791006021279745/5Qke2kf6_normal.jpg'},
  'interactionStatistic': [{'@type': 'InteractionCounter',
    'interactionType': 'https://schema.org/FollowAction',
    'name': 'Follows',
    'userInteractionCount': 1224},
   {'@type': 'InteractionCounter',
    'interactionType': 'https://schema.org/SubscribeAction',
    'name': 'Friends',
    'userInteractionCount': 4952},
   {'@type': 'Intera

In [112]:
user_id = res['author']['identifier']
screenname = res['author']['additionalName']
try:
    name = res['author']['givenName']
except:
    name = "NULL"
followers_count = res['author']['interactionStatistic'][0]['userInteractionCount']
following_count = res['author']['interactionStatistic'][1]['userInteractionCount']
tweet_count = res['author']['interactionStatistic'][2]['userInteractionCount']
try:
    user_bio = res['author']['description']
except:
    user_bio = "NULL"
try:
    bio_link = res['relatedLink'][1]
except:
    bio_link = "NULL"
date_joined = res['dateCreated']
try:
    location = res['author']['homeLocation']['name']
except:
    location = "NULL"

In [113]:
data = {
        "user id" : user_id,
        "screen name" : screenname,
        "name" : name,
        "followers" : followers_count,
        "following" : following_count,
        "bio" : user_bio,
        "bio link" : bio_link,
        "date joined" : date_joined,
        "location" : location,
        "is private" : is_private,
        "is verified" : is_verified,
        "is bussiness account" : is_bussiness_acc,
        "bussiness sector" : bussiness_sector,
}

In [114]:
json_object = json.dumps(data, indent = 4) 
print(json_object)

{
    "user id": "2788849409",
    "screen name": "nuvoforex",
    "name": "Edge Forex",
    "followers": 1224,
    "following": 4952,
    "bio": "We are FOREX signal service provider,who will work hard to get you the best results,so you don't have to.",
    "bio link": "https://telegram.me/+hTMSInfl-Tw2NzU0",
    "date joined": "2014-09-28T11:32:37.000Z",
    "location": "Toronto, Canada",
    "is private": false,
    "is verified": false,
    "is bussiness account": true,
    "bussiness sector": "Financial Tech Company"
}


In [ ]:
def scrape_twitter(driver,accs):
    global master_dict
    master_dict = dict()
    iter_counter = 0
    
    for username in tqdm(accs):
        if iter_counter>50:
            iter_counter = 0 
            time.sleep(2)
        iter_counter+=1
        
        try:
            
            driver.get(f"https://twitter.com/{username}")
            
            try:
                WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, "(//span[text()='Follow']/ancestor::div[@role='button'])[1]")))
            except:
                WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, "(//span[text()='Following']/ancestor::div[@role='button'])[1]")))
                
            html_source = driver.page_source
            sourcedata = html_source.encode('utf-8')
            soup = BeautifulSoup( sourcedata , 'html.parser')
            
            user_id = res['author']['identifier']
            screenname = res['author']['additionalName']
            try:
                name = res['author']['givenName']
            except:
                name = "NULL"
            followers_count = res['author']['interactionStatistic'][0]['userInteractionCount']
            following_count = res['author']['interactionStatistic'][1]['userInteractionCount']
            tweet_count = res['author']['interactionStatistic'][2]['userInteractionCount']
            try:
                user_bio = res['author']['description']
            except:
                user_bio = "NULL"
            try:
                bio_link = res['relatedLink'][1]
            except:
                bio_link = "NULL"
            date_joined = res['dateCreated']
            try:
                location = res['author']['homeLocation']['name']
            except:
                location = "NULL"

            is_private = False
            if soup.find("div",{"class":"css-901oao r-1nao33i r-37j5jr r-1yjpyg1 r-1vr29t4 r-ueyrd6 r-5oul0u r-bcqeeo r-fdjqy7 r-qvutc0"}):
                is_private = True


            is_verified = False
            if soup.find("svg",{"aria-label":"Verified account"}):
                is_verified = True

            is_bussiness_acc = False ; bussiness_sector = "NULL"
            if soup.find("span",{"data-testid":"UserProfessionalCategory"}):
                __ = soup.find("span",{"data-testid":"UserProfessionalCategory"})
                bussiness_sector = __.find("span",{'class':'css-901oao css-16my406 r-poiln3 r-bcqeeo r-qvutc0'}).text
                is_bussiness_acc = True
                
            data = {

                "user id" : user_id,
                "screen name" : screenname,
                "name" : name,
                "followers" : followers_count,
                "following" : following_count,
                "bio" : user_bio,
                "bio link" : bio_link,
                "date joined" : date_joined,
                "location" : location,
                "is private" : is_private,
                "is verified" : is_verified,
                "is bussiness account" : is_bussiness_acc,
                "bussiness sector" : bussiness_sector,

            }
            master_dict.update({screenname:data})
            
            tdf = pd.DataFrame(data,index=[0])
            global df
            df = pd.concat([df,tdf], ignore_index = True)
        
        except TimeoutException:
            df.to_excel("twitter_scraped.xlsx")
            pass
        except IndexError:
            mdf.to_excel("twitter_scraped.xlsx")
            pass
        except TypeError:
            df.to_excel("twitter_scraped.xlsx")
            pass
        finally:
            df.to_excel("twitter_scraped.xlsx")